In [1]:
import pandas as pd

df = pd.read_csv("data/preprocessed_data/preprocessed_calories.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Duration    14999 non-null  float64
 1   Body_Temp   14999 non-null  float64
 2   Heart_Rate  14999 non-null  float64
 3   Age         14999 non-null  int64  
 4   Gender      14999 non-null  object 
 5   Calories    14999 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 703.2+ KB


In [3]:
from pycaret.regression import *

regression_setup = setup(
    data=df,
    target='Calories',
    normalize=True,
    transformation=True,
    session_id=42,
    verbose=True,
    train_size=0.8,
    fold=10,
    fold_shuffle=True,
)

,Description,Value
0,Session id,42
1,Target,Calories
2,Target type,Regression
3,Original data shape,"(14999, 6)"
4,Transformed data shape,"(14999, 6)"
5,Transformed train set shape,"(11999, 6)"
6,Transformed test set shape,"(3000, 6)"
7,Numeric features,4
8,Categorical features,1
9,Preprocess,True


In [4]:
top_models = compare_models(n_select=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:48:14
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,2.4511,12.7545,3.5677,0.9967,0.0532,0.0383,0.0720
et,Extra Trees Regressor,2.5826,14.5438,3.8108,0.9963,0.0516,0.0375,0.1850
rf,Random Forest Regressor,2.5910,14.6950,3.8318,0.9962,0.0523,0.0380,0.2080
gbr,Gradient Boosting Regressor,3.2907,20.6265,4.5367,0.9947,0.1132,0.0691,0.0670
knn,K Neighbors Regressor,3.6082,26.5041,5.1433,0.9932,0.0827,0.0616,0.0230
dt,Decision Tree Regressor,3.7545,32.1146,5.6660,0.9918,0.0739,0.0523,0.0190
ada,AdaBoost Regressor,9.5307,142.2284,11.9170,0.9636,0.4269,0.4441,0.0640
lar,Least Angle Regression,11.5139,221.4260,14.8744,0.9433,0.4675,0.5192,0.0190
br,Bayesian Ridge,11.5140,221.4261,14.8744,0.9433,0.4675,0.5192,0.0180
ridge,Ridge Regression,11.5141,221.4262,14.8744,0.9433,0.4675,0.5192,0.0180


In [5]:
tuned_models = [tune_model(m) for m in top_models]

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:48:34
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.4129,12.5105,3.5370,0.9966,0.0535,0.0384
1,2.5602,13.5239,3.6775,0.9965,0.0558,0.0398
2,2.5337,13.2134,3.6350,0.9967,0.0637,0.0428
3,2.3941,11.8183,3.4378,0.9969,0.0503,0.0372
4,2.3838,11.6284,3.4100,0.9970,0.0551,0.0376
5,2.3242,10.7892,3.2847,0.9971,0.0572,0.0397
6,2.3637,11.6434,3.4122,0.9971,0.0508,0.0370
7,2.4464,12.4356,3.5264,0.9967,0.0684,0.0414
8,2.6763,14.6468,3.8271,0.9964,0.0651,0.0436


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:48:45
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.7853,51.4455,7.1726,0.9860,0.0988,0.0764
1,4.7982,49.2999,7.0214,0.9873,0.1103,0.0837
2,4.6069,46.1709,6.7949,0.9884,0.1060,0.0798
3,4.3705,39.9127,6.3177,0.9897,0.0972,0.0743
4,4.5722,45.2145,6.7242,0.9882,0.1082,0.0808
5,4.6670,44.4732,6.6688,0.9880,0.1030,0.0789
6,4.4375,43.5948,6.6026,0.9893,0.0973,0.0747
7,4.4097,41.3954,6.4339,0.9891,0.1121,0.0850
8,5.1712,58.1084,7.6229,0.9859,0.1111,0.0833


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:50:27
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.0923,23.9821,4.8972,0.9935,0.0667,0.0480
1,3.0479,20.6400,4.5431,0.9947,0.0751,0.0515
2,3.1000,20.5972,4.5384,0.9948,0.0686,0.0495
3,2.9059,18.6623,4.3200,0.9952,0.0621,0.0456
4,2.9714,18.3824,4.2875,0.9952,0.0700,0.0496
5,2.7995,16.6391,4.0791,0.9955,0.0659,0.0470
6,2.8851,18.5847,4.3110,0.9954,0.0650,0.0465
7,2.9614,19.1676,4.3781,0.9950,0.0728,0.0513
8,3.3460,24.4518,4.9449,0.9941,0.0762,0.0532


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [6]:
best_tuned_model = compare_models(tuned_models, sort='R2')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:52:34
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Light Gradient Boosting Machine,2.4572,12.5454,3.5387,0.9968,0.0577,0.0398,0.0860
1,Extra Trees Regressor,2.5826,14.5438,3.8108,0.9963,0.0516,0.0375,0.2070
2,Random Forest Regressor,2.5910,14.6950,3.8318,0.9962,0.0523,0.0380,0.2070


In [14]:
evaluate_model(best_tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [8]:
final_model = finalize_model(best_tuned_model)

In [9]:
save_model(final_model, 'final_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Duration', 'Body_Temp',
                                              'Heart_Rate', 'Age'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['Gender'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('ordinal_encoding',
                  TransformerWrapper(include=['Gender'],
                                     transforme...
                  TransformerWrapper(transformer=PowerTransformer(standardize=False))),
                 ('normalize', TransformerWrapper(transformer=StandardScaler())),
                 ('actual_estimator',
                  LGBMRegressor(bagging_fraction=0.8, bagging_freq=3,
                                feature_fraction=0.8, learning_rate=0.2,
                           

In [10]:
predict_model(best_tuned_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,2.4300,12.6913,3.5625,0.9967,0.0549,0.0381


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


,Duration,Body_Temp,Heart_Rate,Age,Gender,Calories,prediction_label
6723,27.0,41.200001,111.0,41,male,203.0,197.057938
6473,26.0,40.900002,113.0,68,male,223.0,237.105110
4679,2.0,38.400002,77.0,50,female,6.0,6.149969
862,24.0,40.799999,108.0,35,male,161.0,159.614198
7286,19.0,40.500000,97.0,22,male,86.0,82.697902
...,...,...,...,...,...,...,...
3297,14.0,40.400002,91.0,74,female,72.0,75.522712
14113,6.0,39.299999,86.0,23,male,16.0,16.065291
5514,7.0,39.700001,86.0,55,male,34.0,30.951761
9939,11.0,39.599998,101.0,30,male,61.0,57.630566
